In [6]:
import ctypes, pywintypes as wt
from ctypes import *
from win32con import GENERIC_READ, GENERIC_WRITE, OPEN_EXISTING, FILE_SHARE_READ, FILE_SHARE_WRITE

BOOL = c_int
LPVOID = c_void_p
DWORD = c_ulong
LPDWORD = POINTER(DWORD)
HANDLE = c_void_p
LPCHAR = POINTER(c_char)

CloseHandle = windll.kernel32.CloseHandle

_rft = ctypes.CFUNCTYPE(BOOL, HANDLE, LPVOID, DWORD, LPDWORD, LPVOID)
_read_file = _rft(windll.kernel32.ReadFile)

def read_file(hfile, nbytes):
    buf = (c_char*nbytes)()
    lpread = LPDWORD()
    lpread.value = 0
    overlapped = None
    res = _read_file(hfile, buf, nbytes, lpread, overlapped)
    if res == 0:
        raise OSError(windll.kernel32.GetLastError())
    return buf.value

_wft = ctypes.CFUNCTYPE(BOOL, HANDLE, LPVOID, DWORD, LPDWORD, LPVOID)
_write_file = _wft(windll.kernel32.WriteFile)
def write_file(hfile, buf, n=None):
    if n is None:
        n = len(buf)
    lpwritten = LPDWORD()
    lpwritten.value = 0
    overlapped=None
    res = _write_file(hfile, buf, n, lpwritten, overlapped)
    if res == 0: 
        raise OSError(windll.kernel32.GetLastError())
    return lpwritten.value

_cft = ctypes.CFUNCTYPE(HANDLE, LPCHAR, DWORD, DWORD, LPVOID, DWORD, DWORD, HANDLE)
_create_file = _cft(windll.kernel32.CreateFileA)
def create_file(filename, access=GENERIC_READ|GENERIC_WRITE, share=0, sec=None, disp=OPEN_EXISTING, attrib=0, htmpl=None):
    if not isinstance(filename, bytes):
        filename = filename.encode('ascii')
    rv = _create_file(filename, access, share, sec, disp, attrib, htmpl)
    if rv == 4294967295:
        raise OSError(windll.kernel32.GetLastError())
    return rv
    

In [7]:
class M10Driver():
    def __init__(self, port="\\\\.\\COM4"):
        self.port = port
        self.handle = create_file(port)
        
    def write(self, msg):
        if not isinstance(msg, bytes):
            msg = msg.encode('ascii')
        return write_file(self.handle, msg)
    
    def read(self, n=128):
        return read_file(self.handle, n)
    
    def close(self, _close=CloseHandle):
        if self.handle:
            _close(self.handle)
        self.handle = None
    
    def __del__(self):
        self.close()
    
    
class M10Gauge():
    def __init__(self, port_or_driver):
        if isinstance(port_or_driver, M10Driver):
            self.driver = port_or_driver
        else:
            self.driver = M10Driver(port)
        self._buf = []
        
    def readline(self):
        if len(self._buf) <=1:
            self._read_driver(1024)
        return self._buf.pop(0)
        
    def _read_driver(self, n):
        buf = self.driver.read(n)
        if self._buf:
            b2 = self._buf.pop()
            b2.extend(buf)
        else:
            b2 = bytearray(buf)
        lines = buf.split(b'\r\n')
        self._buf.extend(lines)
        
    def read_display(self):
        self.driver.write(b"?\r\n")
        msg = b''
        while not msg.endswith(b'\r\n'):
            
            msg += self.driver.read()
           

In [8]:
def read_forever():
    from time import sleep
    g = M10Gauge(M10Driver())
    try:
        while True:
            d = g.read_display()
            print("\r%s       \r" % d, end="")
            sleep(0.5)
    finally:
        g.driver.close()
        
    

In [9]:
#read_forever()

In [10]:
g = M10Gauge(M10Driver())

In [11]:
h = g.driver.handle

In [12]:
n = 1
buf = c_char * n
buf = buf()
read = DWORD()
_read_file(h, buf, n, byref(read), None)

1

In [13]:
def ReadFile(hfile, n):
    buf = (c_char * n)()
    read = DWORD()
    ovl = None
    rv= _read_file(hfile, buf, n, byref(read), ovl)
    return buf.value

In [17]:
ReadFile(h, 10)

b''